1) Load Google Colab, Mount the Directory.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


2) Inspect the Source - Number of Images Per Class, Sizes of Images Per Class, Channels of Images Per Class, and Total Number of Images.

In [ ]:
import os
from PIL import Image
from collections import Counter, defaultdict

# ── Configuration ──────────────────────────────────────────────────────────
src_dir = "/content/drive/MyDrive/Research Project 2025/Preprocessed Datasets/Samples/Source - MedMNIST Labelled/Images"
# ── End configuration ──────────────────────────────────────────────────────

def inspect_directory(path):
    class_counts = Counter()
    class_modes  = defaultdict(Counter)
    class_res    = defaultdict(Counter)

    for cls in sorted(os.listdir(path)):
        cls_path = os.path.join(path, cls)
        if not os.path.isdir(cls_path):
            continue
        for fname in os.listdir(cls_path):
            if not fname.lower().endswith(('.png','.jpg','.jpeg','.bmp','.tiff')):
                continue
            class_counts[cls] += 1
            img_path = os.path.join(cls_path, fname)
            try:
                with Image.open(img_path) as img:
                    class_modes[cls][img.mode] += 1
                    class_res[cls][img.size] += 1
            except Exception as e:
                print(f"❌ Error opening {img_path}: {e}")

    total = sum(class_counts.values())
    print(f"\nInspection of `{path}`")
    print(f"{'Class':<12}  Images  Modes               Top Resolutions")
    print("-"*60)
    for cls, cnt in class_counts.items():
        modes_s = ", ".join(f"{m}:{n}" for m,n in class_modes[cls].items())
        top3    = class_res[cls].most_common(3)
        res_s   = ", ".join(f"{w}×{h}:{n}" for (w,h),n in top3)
        print(f"{cls:<12}  {cnt:<6}  {modes_s:<18}  {res_s}")
    print(f"\n→ Total images: {total}\n")

# Run the inspection
inspect_directory(src_dir)


Inspection of `/content/drive/MyDrive/Research Project 2025/Preprocessed Datasets/Samples/Source - MedMNIST Labelled/Images`
Class         Images  Modes               Top Resolutions
------------------------------------------------------------
AbdomenCT     130     RGB:130             224×224:130
BreastMRI     130     RGB:130             224×224:130
CXR           130     RGB:130             224×224:130
ChestCT       130     RGB:130             224×224:130
HandXR        130     RGB:130             224×224:130
HeadCT        130     RGB:130             224×224:130

→ Total images: 780



3) Inspect the Target (unlabelled) dataset if it has 780 images, 224x224 size, and 3 channels.

In [ ]:
import os
from PIL import Image
from collections import Counter

# ── Configuration ──────────────────────────────────────────────────────────
unlabelled_dir = "/content/drive/MyDrive/Research Project 2025/Preprocessed Datasets/Samples/Target - VS Unlabelled/Images"
# ── End configuration ──────────────────────────────────────────────────────

# Counters for total, modes, and resolutions
total_images = 0
mode_counts  = Counter()
size_counts  = Counter()

# Iterate through all images
for fname in os.listdir(unlabelled_dir):
    if not fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        continue
    total_images += 1
    img_path = os.path.join(unlabelled_dir, fname)
    try:
        with Image.open(img_path) as img:
            mode_counts[img.mode] += 1
            size_counts[img.size] += 1
    except Exception as e:
        print(f"❌ Error opening {img_path}: {e}")

# Display results
print(f"\nInspection of unlabelled target directory: `{unlabelled_dir}`")
print(f"→ Total images: {total_images}\n")

print("Channel modes:")
for mode, cnt in mode_counts.items():
    print(f"  {mode}: {cnt}")

print("\nImage resolutions:")
for (w, h), cnt in size_counts.most_common():
    print(f"  {w}×{h}: {cnt}")


Inspection of unlabelled target directory: `/content/drive/MyDrive/Research Project 2025/Preprocessed Datasets/Samples/Target - VS Unlabelled/Images`
→ Total images: 780

Channel modes:
  RGB: 780

Image resolutions:
  224×224: 780


4) Let us Normalize the source.

In [ ]:
import os
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm

# ── Configuration ────────────────────────────────────────────────────────────
data_dir = "/content/drive/MyDrive/Research Project 2025/Preprocessed Datasets/Samples/Source - MedMNIST Labelled/Images"
batch_size = 64   # for batch-level stats
num_workers = 4
# ── End configuration ─────────────────────────────────────────────────────────

# 1) Pre-normalization: load as Tensor [0,1] but no Normalize
pre_transform = transforms.Compose([
    transforms.ToTensor()   # converts to [C,H,W] in [0.0,1.0]
])
pre_ds = datasets.ImageFolder(data_dir, transform=pre_transform)
pre_loader = DataLoader(pre_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# Compute channel sums & squared sums
n_channels = 3
cnt = 0
sum_ = torch.zeros(n_channels)
sum_sq = torch.zeros(n_channels)

for imgs, _ in tqdm(pre_loader, desc="Pre-norm stats"):
    # imgs shape: [B, C, H, W]
    b, c, h, w = imgs.shape
    cnt += b * h * w
    sum_ += imgs.sum(dim=[0,2,3])
    sum_sq += (imgs ** 2).sum(dim=[0,2,3])

mean_pre = sum_ / cnt
var_pre = (sum_sq / cnt) - (mean_pre ** 2)
std_pre = torch.sqrt(var_pre)

print("\nPre-normalization mean:", mean_pre)
print("Pre-normalization std: ", std_pre)


# 2) One-batch post-normalization: take first batch with Normalize()
normalize = transforms.Normalize(mean=mean_pre.tolist(), std=std_pre.tolist())
batch_transform = transforms.Compose([
    transforms.ToTensor(),
    normalize
])
batch_ds = datasets.ImageFolder(data_dir, transform=batch_transform)
batch_loader = DataLoader(batch_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# Get one batch
imgs_batch, _ = next(iter(batch_loader))  # [B, C, H, W]
mean_batch = imgs_batch.mean(dim=[0,2,3])
std_batch  = imgs_batch.std(dim=[0,2,3])

print("\nOne-batch post-norm mean:", mean_batch)
print("One-batch post-norm std: ", std_batch)


# 3) Full-dataset post-normalization: entire loader with Normalize
post_loader = DataLoader(batch_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers)

cnt2 = 0
sum2 = torch.zeros(n_channels)
sum2_sq = torch.zeros(n_channels)

for imgs, _ in tqdm(post_loader, desc="Post-norm full stats"):
    b, c, h, w = imgs.shape
    cnt2 += b * h * w
    sum2 += imgs.sum(dim=[0,2,3])
    sum2_sq += (imgs ** 2).sum(dim=[0,2,3])

mean_post = sum2 / cnt2
var_post = (sum2_sq / cnt2) - (mean_post ** 2)
std_post = torch.sqrt(var_post)

print("\nPost-normalization mean:", mean_post)
print("Post-normalization std: ", std_post)

Pre-norm stats: 100%|██████████| 13/13 [02:33<00:00, 11.81s/it]


Pre-normalization mean: tensor([0.3542, 0.3542, 0.3542])
Pre-normalization std:  tensor([0.2794, 0.2794, 0.2794])



One-batch post-norm mean: tensor([0.3144, 0.3144, 0.3144])
One-batch post-norm std:  tensor([0.3413, 0.3413, 0.3413])


Post-norm full stats: 100%|██████████| 13/13 [00:01<00:00,  8.32it/s]


Post-normalization mean: tensor([3.1939e-08, 3.1939e-08, 3.1939e-08])
Post-normalization std:  tensor([1.0000, 1.0000, 1.0000])


5) Let us build the EfficientNet architecture.
Inspired from : https://arxiv.org/pdf/1905.11946

In [ ]:
# EfficientNet-B0 using PyTorch + summary
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

# ---- Utilities ----
def _make_divisible(v, divisor=8, min_value=None):
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    if new_v < 0.9 * v:  # ensure round does not go down by >10%
        new_v += divisor
    return new_v

class SqueezeExcite(nn.Module):
    def __init__(self, in_chs, se_ratio=0.25):
        super().__init__()
        reduced = _make_divisible(in_chs * se_ratio, 8)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.reduce = nn.Conv2d(in_chs, reduced, 1, bias=True)
        self.expand = nn.Conv2d(reduced, in_chs, 1, bias=True)

    def forward(self, x):
        s = self.pool(x)
        s = F.silu(self.reduce(s))  # SiLU/Swish
        s = torch.sigmoid(self.expand(s))
        return x * s

def drop_connect(x, drop_rate, training):
    if not training or drop_rate == 0.0:
        return x
    keep = 1.0 - drop_rate
    mask = torch.empty((x.shape[0], 1, 1, 1), dtype=x.dtype, device=x.device).bernoulli_(keep)
    return x / keep * mask

# ---- MBConv (Mobile inverted bottleneck with SE) ----
class MBConv(nn.Module):
    def __init__(self, in_chs, out_chs, kernel_size, stride, expand_ratio, se_ratio=0.25, drop_connect_rate=0.0):
        super().__init__()
        self.stride = stride
        self.in_chs = in_chs
        self.out_chs = out_chs
        self.has_residual = (stride == 1 and in_chs == out_chs)
        mid_chs = in_chs * expand_ratio

        layers = []
        # 1) Expansion (1x1) if t > 1
        if expand_ratio != 1:
            layers += [
                nn.Conv2d(in_chs, mid_chs, 1, bias=False),
                nn.BatchNorm2d(mid_chs),
                nn.SiLU(inplace=True),
            ]
        # 2) Depthwise conv (k x k)
        layers += [
            nn.Conv2d(mid_chs, mid_chs, kernel_size, stride=stride,
                      padding=kernel_size // 2, groups=mid_chs, bias=False),
            nn.BatchNorm2d(mid_chs),
            nn.SiLU(inplace=True),
        ]
        # 3) Squeeze-and-Excitation
        layers += [SqueezeExcite(mid_chs, se_ratio=se_ratio)]
        # 4) Projection (1x1, linear)
        layers += [
            nn.Conv2d(mid_chs, out_chs, 1, bias=False),
            nn.BatchNorm2d(out_chs),
        ]
        self.block = nn.Sequential(*layers)
        self.drop_connect_rate = drop_connect_rate

    def forward(self, x):
        out = self.block(x)
        if self.has_residual:
            if self.drop_connect_rate:
                out = drop_connect(out, self.drop_connect_rate, self.training)
            out = out + x
        return out

# ---- EfficientNet-B0 ----
class EfficientNetB0(nn.Module):
    """
    B0:
      - Stem: 32, 3x3, s=2
      - Blocks (r, k, s, c, t) per paper's Table 1
      - SE ratio: 0.25
      - Head: 1x1 conv to 1280, GAP, dropout=0.2, FC
    """
    def __init__(self, num_classes=6, drop_rate=0.2, drop_connect_rate=0.2):
        super().__init__()
        self.drop_rate = drop_rate
        self.drop_connect_rate = drop_connect_rate
        se_ratio = 0.25

        # Stem
        self.stem = nn.Sequential(
            nn.Conv2d(3, 32, 3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.SiLU(inplace=True),
        )

        # Block settings for B0: (repeats, k, s, out_c, expand)
        cfgs = [
            (1, 3, 1,  16, 1),
            (2, 3, 2,  24, 6),
            (2, 5, 2,  40, 6),
            (3, 3, 2,  80, 6),
            (3, 5, 1, 112, 6),
            (4, 5, 2, 192, 6),
            (1, 3, 1, 320, 6),
        ]

        blocks = []
        in_chs = 32
        total_blocks = sum(r for r, *_ in cfgs)
        b_idx = 0
        for (repeats, k, s, out_c, t) in cfgs:
            for i in range(repeats):
                stride = s if i == 0 else 1
                # linearly scale drop_connect across blocks
                dcr = self.drop_connect_rate * b_idx / max(1, total_blocks - 1)
                blocks.append(MBConv(in_chs, out_c, kernel_size=k, stride=stride,
                                     expand_ratio=t, se_ratio=se_ratio, drop_connect_rate=dcr))
                in_chs = out_c
                b_idx += 1

        self.blocks = nn.Sequential(*blocks)

        # Head
        self.head = nn.Sequential(
            nn.Conv2d(in_chs, 1280, 1, bias=False),
            nn.BatchNorm2d(1280),
            nn.SiLU(inplace=True),
        )
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(1280, num_classes)

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight); nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01); nn.init.zeros_(m.bias)

    # handy for DA heads
    def forward_features(self, x):
        x = self.stem(x)
        x = self.blocks(x)
        x = self.head(x)
        return self.pool(x).flatten(1)     # 1280-d

    def forward(self, x):
        x = self.forward_features(x)
        if self.drop_rate:
            x = F.dropout(x, p=self.drop_rate, training=self.training)
        x = self.classifier(x)
        return x

# --- Smoke test + Summary ---
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    m = EfficientNetB0(num_classes=6)

    # try to import torchinfo; auto-install if missing (Colab-friendly)
    try:
        from torchinfo import summary
    except Exception as e:
        import sys, subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "torchinfo"])
        from torchinfo import summary

    # Print Keras-like summary
    print(summary(
        m,
        input_size=(1, 3, 224, 224),
        device=device,
        col_names=("input_size", "output_size", "num_params", "kernel_size"),
        depth=3,    # enough to expand MBConv internals
        verbose=1
    ))

    # Quick forward check
    x = torch.randn(2, 3, 224, 224, device=device)
    y = m(x)
    print("Output shape:", y.shape)  # expect: torch.Size([2, 6])

Layer (type:depth-idx)                             Input Shape               Output Shape              Param #                   Kernel Shape
EfficientNetB0                                     [1, 3, 224, 224]          [1, 6]                    --                        --
├─Sequential: 1-1                                  [1, 3, 224, 224]          [1, 32, 112, 112]         --                        --
│    └─Conv2d: 2-1                                 [1, 3, 224, 224]          [1, 32, 112, 112]         864                       [3, 3]
│    └─BatchNorm2d: 2-2                            [1, 32, 112, 112]         [1, 32, 112, 112]         64                        --
│    └─SiLU: 2-3                                   [1, 32, 112, 112]         [1, 32, 112, 112]         --                        --
├─Sequential: 1-2                                  [1, 32, 112, 112]         [1, 320, 7, 7]            --                        --
│    └─MBConv: 2-4                                 [1, 32, 112

6) Let us prepare the data loaders for split.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os, torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset

# Paths
source_dir = '/content/drive/MyDrive/Research Project 2025/Preprocessed Datasets/Samples/Source - MedMNIST Labelled/Images'

# Precomputed normalization stats
mean = [0.3542, 0.3542, 0.3542]
std  = [0.2794, 0.2794, 0.2794]

# Transforms
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(8),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])
val_test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

# 1) Load a base dataset (no transform) just to get length/labels
base_ds = ImageFolder(source_dir, transform=None)

# 2) Deterministic split indices (NOT random_split on a list)
torch.manual_seed(1906525)
n_total = len(base_ds)
n_train = int(0.75 * n_total)
n_val   = int(0.10 * n_total)
n_test  = n_total - n_train - n_val

perm = torch.randperm(n_total)
train_idx = perm[:n_train]
val_idx   = perm[n_train:n_train+n_val]
test_idx  = perm[n_train+n_val:]

# 3) Make three datasets with their own transforms, then subset by the same indices
train_full = ImageFolder(source_dir, transform=train_transform)
val_full   = ImageFolder(source_dir, transform=val_test_transform)
test_full  = ImageFolder(source_dir, transform=val_test_transform)

train_ds = Subset(train_full, train_idx)
val_ds   = Subset(val_full,   val_idx)
test_ds  = Subset(test_full,  test_idx)

# 4) DataLoaders (Drive I/O tips: pin_memory + persistent_workers)
batch_size, num_workers = 32, 4
loader_kwargs = dict(batch_size=batch_size, num_workers=num_workers,
                     pin_memory=True, persistent_workers=(num_workers > 0))

train_loader = DataLoader(train_ds, shuffle=True,  **loader_kwargs)
val_loader   = DataLoader(val_ds,   shuffle=False, **loader_kwargs)
test_loader  = DataLoader(test_ds,  shuffle=False, **loader_kwargs)

# 5) Sanity check
print("Classes (alphabetical order):", base_ds.classes)  # label mapping in .class_to_idx
print("class_to_idx:", base_ds.class_to_idx)
print(f"Train size: {len(train_ds)}, Val size: {len(val_ds)}, Test size: {len(test_ds)}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Classes (alphabetical order): ['AbdomenCT', 'BreastMRI', 'CXR', 'ChestCT', 'HandXR', 'HeadCT']
class_to_idx: {'AbdomenCT': 0, 'BreastMRI': 1, 'CXR': 2, 'ChestCT': 3, 'HandXR': 4, 'HeadCT': 5}
Train size: 585, Val size: 78, Test size: 117


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


7) Let us start modelling for the source MedMNIST dataset samples with EfficientNet and save the checkpoints.

In [ ]:
# ─── 0) Mount & Imports ─────────────────────────────────────────────────────
from google.colab import drive
drive.mount('/content/drive')

import os, random, numpy as np
import torch, torch.nn as nn, torch.backends.cudnn as cudnn
import pandas as pd, seaborn as sns, matplotlib.pyplot as plt

from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset
from torch.optim import SGD, Adam, RMSprop, Adagrad, AdamW
import torch.optim.lr_scheduler as lr_scheduler

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, mean_squared_error, confusion_matrix
)

# NOTE: assumes EfficientNetB0 class is already defined above in this notebook.

# ─── 1) Determinism ─────────────────────────────────────────────────────────
SEED = 1906525
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
cudnn.deterministic = True   # deterministic conv algorithms
cudnn.benchmark = False      # avoid non-deterministic autotuner
# See: https://pytorch.org/docs/stable/notes/randomness.html  (reproducibility notes)

# ─── 2) Paths & Hyperparams ─────────────────────────────────────────────────
SRC_DIR = '/content/drive/MyDrive/Research Project 2025/Preprocessed Datasets/Samples/Source - MedMNIST Labelled/Images'

CKPT_DIR    = '/content/drive/MyDrive/Research Project 2025/Results/Samples/EfficientNet/Source Training/Checkpoints'
CM_DIR      = '/content/drive/MyDrive/Research Project 2025/Results/Samples/EfficientNet/Source Training/Confusion Matrices'
METRICS_DIR = '/content/drive/MyDrive/Research Project 2025/Results/Samples/EfficientNet/Source Training/Performance Metrics'
os.makedirs(CKPT_DIR, exist_ok=True)
os.makedirs(CM_DIR, exist_ok=True)
os.makedirs(METRICS_DIR, exist_ok=True)

MEAN = [0.3542, 0.3542, 0.3542]
STD  = [0.2794, 0.2794, 0.2794]

train_tf = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(8),
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD),
])
val_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD),
])

OPTIMIZERS  = [SGD, Adam, RMSprop, Adagrad, AdamW]
BATCH_SIZE  = 32
EPOCHS_LIST = [5, 10, 20, 40, 80]
LR          = 1e-3
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ─── 3) Prepare DataLoaders (deterministic split, per‑split transforms) ─────
base_ds = ImageFolder(SRC_DIR, transform=None)  # labels inferred by folder names
# ImageFolder docs: has .classes and .class_to_idx  (for sanity printing)
# https://pytorch.org/vision/main/generated/torchvision.datasets.ImageFolder.html

n_total = len(base_ds)
n_train = int(0.75 * n_total)
n_val   = int(0.10 * n_total)
n_test  = n_total - n_train - n_val

g = torch.Generator().manual_seed(SEED)
perm = torch.randperm(n_total, generator=g)
train_idx = perm[:n_train]
val_idx   = perm[n_train:n_train+n_val]
test_idx  = perm[n_train+n_val:]

train_full = ImageFolder(SRC_DIR, transform=train_tf)
val_full   = ImageFolder(SRC_DIR, transform=val_tf)
test_full  = ImageFolder(SRC_DIR, transform=val_tf)

train_ds = Subset(train_full, train_idx)
val_ds   = Subset(val_full,   val_idx)
test_ds  = Subset(test_full,  test_idx)

loader_kwargs = dict(batch_size=BATCH_SIZE, num_workers=4, pin_memory=True)
train_loader = DataLoader(train_ds, shuffle=True,  **loader_kwargs)
val_loader   = DataLoader(val_ds,   shuffle=False, **loader_kwargs)
test_loader  = DataLoader(test_ds,  shuffle=False, **loader_kwargs)

print("Classes:", base_ds.classes)           # alphabetical
print("class_to_idx:", base_ds.class_to_idx) # mapping folder -> label

# ─── 4) Metrics / Eval helper ───────────────────────────────────────────────
def evaluate_split(model, loader, criterion=None, plot_cm=False, cm_name=None):
    model.eval()
    all_preds, all_labels, all_probs = [], [], []
    total_loss, total_samples = 0.0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            logits = model(x)
            if criterion is not None:
                total_loss += criterion(logits, y).item() * x.size(0)
            probs = torch.softmax(logits, dim=1).cpu().numpy()
            preds = logits.argmax(1).cpu().numpy()
            all_probs.extend(probs)
            all_preds.extend(preds)
            all_labels.extend(y.cpu().numpy())
            total_samples += x.size(0)

    acc = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    rec = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    # Multiclass AUC via one-vs-one (ovo); requires probability scores
    try:
        auc = roc_auc_score(all_labels, np.array(all_probs), multi_class='ovo')
    except Exception:
        auc = float('nan')  # e.g., single-class batch edge cases
    mse = mean_squared_error(all_labels, all_preds)

    # Specificity (macro over classes) from confusion matrix
    cm = confusion_matrix(all_labels, all_preds, labels=list(range(len(loader.dataset.dataset.classes))))
    total = cm.sum()
    specs = []
    for i in range(cm.shape[0]):
        tp = cm[i, i]
        fn = cm[i, :].sum() - tp
        fp = cm[:, i].sum() - tp
        tn = total - tp - fp - fn
        specs.append(tn / (tn + fp) if (tn + fp) > 0 else 0.0)
    spec = float(np.mean(specs))

    if plot_cm and cm_name:
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=loader.dataset.dataset.classes,
                    yticklabels=loader.dataset.dataset.classes)
        plt.xlabel('Predicted'); plt.ylabel('Actual')
        plt.tight_layout()
        plt.savefig(os.path.join(CM_DIR, cm_name))
        plt.close()

    loss = (total_loss / total_samples) if criterion is not None else None
    return acc, prec, rec, spec, f1, auc, mse, loss

# AUC API ref: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
# Multiclass ROC example: https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html

# ─── 5) Training & Logging ─────────────────────────────────────────────────
master = []

for opt_cls in OPTIMIZERS:
    for num_epochs in EPOCHS_LIST:
        run_name = f"{opt_cls.__name__}_ep{num_epochs}"
        print(f"\n▶ Run: {run_name}")

        model = EfficientNetB0(num_classes=len(base_ds.classes)).to(DEVICE)
        criterion = nn.CrossEntropyLoss()
        optimizer = opt_cls(model.parameters(), lr=LR)
        scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

        # gradient norm for curiosity (before & after)
        def get_grad_norm():
            model.eval(); model.zero_grad()
            x, y = next(iter(train_loader))
            x, y = x.to(DEVICE), y.to(DEVICE)
            loss = criterion(model(x), y)
            loss.backward()
            total = 0.0
            for p in model.parameters():
                if p.grad is not None:
                    g = p.grad.detach()
                    total += float(torch.norm(g)**2)
            return total**0.5

        grad_before = get_grad_norm()

        for epoch in range(1, num_epochs + 1):
            model.train()
            for x, y in train_loader:
                x, y = x.to(DEVICE), y.to(DEVICE)
                optimizer.zero_grad()
                loss = criterion(model(x), y)
                loss.backward()
                optimizer.step()

            # validation & LR schedule
            _, _, _, _, _, _, _, val_loss = evaluate_split(model, val_loader, criterion)
            scheduler.step(val_loss)

            # mid‑run CM at epoch 40
            if epoch == 40:
                _ = evaluate_split(
                    model, test_loader, plot_cm=True, cm_name=f"{opt_cls.__name__}_ep40.png"
                )

        # final metrics on train/val/test
        train_acc, train_prec, train_rec, train_spec, train_f1, train_auc, train_mse, _ = evaluate_split(model, train_loader)
        val_acc, val_prec, val_rec, val_spec, val_f1, val_auc, val_mse, _ = evaluate_split(model, val_loader)
        test_acc, test_prec, test_rec, test_spec, test_f1, test_auc, test_mse, _ = evaluate_split(model, test_loader)
        grad_after = get_grad_norm()

        print(f"Completed {run_name} → Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}")

        # save checkpoint
        torch.save(model.state_dict(), os.path.join(CKPT_DIR, f"{opt_cls.__name__}_ep{num_epochs}.pth"))

        # log row
        master.append({
            'Model': 'EfficientNetB0',
            'Optimizer': opt_cls.__name__, 'Epochs': num_epochs,
            'train_acc': train_acc, 'val_acc': val_acc, 'test_acc': test_acc,
            'train_mse': train_mse, 'val_mse': val_mse, 'test_mse': test_mse,
            'precision': test_prec, 'sensitivity': test_rec,
            'specificity': test_spec, 'f1': test_f1, 'auc': test_auc,
            'grad_before': grad_before, 'grad_after': grad_after
        })

# write metrics Excel
metrics_file = os.path.join(METRICS_DIR, 'EfficientNetB0_sample_source.xlsx')
pd.DataFrame(master).to_excel(metrics_file, index=False)
print("Training completed. Saved checkpoints, confusion matrices, and metrics for EfficientNet‑B0 (source).")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Classes: ['AbdomenCT', 'BreastMRI', 'CXR', 'ChestCT', 'HandXR', 'HeadCT']
class_to_idx: {'AbdomenCT': 0, 'BreastMRI': 1, 'CXR': 2, 'ChestCT': 3, 'HandXR': 4, 'HeadCT': 5}

▶ Run: SGD_ep5
Completed SGD_ep5 → Train Acc: 0.2752, Test Acc: 0.2821

▶ Run: SGD_ep10
Completed SGD_ep10 → Train Acc: 0.4308, Test Acc: 0.4786

▶ Run: SGD_ep20
Completed SGD_ep20 → Train Acc: 0.5983, Test Acc: 0.5897

▶ Run: SGD_ep40
Completed SGD_ep40 → Train Acc: 0.8444, Test Acc: 0.8291

▶ Run: SGD_ep80
Completed SGD_ep80 → Train Acc: 0.9880, Test Acc: 0.9915

▶ Run: Adam_ep5
Completed Adam_ep5 → Train Acc: 0.9624, Test Acc: 0.9744

▶ Run: Adam_ep10
Completed Adam_ep10 → Train Acc: 0.8068, Test Acc: 0.8291

▶ Run: Adam_ep20
Completed Adam_ep20 → Train Acc: 1.0000, Test Acc: 0.9915

▶ Run: Adam_ep40
Completed Adam_ep40 → Train Acc: 1.0000, Test Acc: 0.9915

▶ Run: Adam_ep80
Completed Ad

8) Let us inspect the target (unlabelled) dataset's channels and image sizes.

In [ ]:
import os
from pathlib import Path
from collections import Counter
from PIL import Image

TARGET_DIR = Path("/content/drive/MyDrive/Research Project 2025/Preprocessed Datasets/Samples/Target - VS Unlabelled/Images")

IMAGE_EXTS = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif'}
files = [f for f in TARGET_DIR.rglob("*") if f.suffix.lower() in IMAGE_EXTS and f.is_file()]

print(f"Total image files: {len(files)}")

mode_counter = Counter()
size_counter = Counter()
dimensions = []

for f in files:
    try:
        with Image.open(f) as img:
            mode = img.mode  # e.g., 'RGB', 'L', etc.
            width, height = img.size
            mode_counter[mode] += 1
            size_counter[(width, height)] += 1
            dimensions.append((width, height))
    except Exception as e:
        print(f"Error with {f}: {e}")

print("\nImage modes (counts):")
for mode, count in mode_counter.items():
    print(f"  Mode: {mode}, Count: {count}")

print("\nUnique image dimensions (width × height) and their counts:")
for (w, h), count in size_counter.most_common():
    print(f"  {w}×{h}: {count} images")

# Function to get channels from mode
def channels_from_mode(mode):
    # As noted on StackOverflow, image.getbands() gives accurate channel count
    return len(Image.new(mode, (1,1)).getbands())

print("\nChannels per mode:")
for mode in mode_counter:
    print(f"  Mode: {mode}, Channels: {channels_from_mode(mode)}")

Total image files: 780

Image modes (counts):
  Mode: RGB, Count: 780

Unique image dimensions (width × height) and their counts:
  224×224: 780 images

Channels per mode:
  Mode: RGB, Channels: 3


9) Let us do normalization for the target (unlabelled) dataset.

In [ ]:
import os
from pathlib import Path
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class UnlabeledImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = Path(root_dir)
        self.transform = transform
        self.image_paths = [p for p in self.root_dir.iterdir() if p.suffix.lower() in {'.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif'}]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img

# 1. Define paths
root = "/content/drive/MyDrive/Research Project 2025/Preprocessed Datasets/Samples/Target - VS Unlabelled/Images"

# 2. Base loader to compute pre-normalization mean & std
base_ds = UnlabeledImageDataset(root, transform=transforms.ToTensor())
base_loader = DataLoader(base_ds, batch_size=64, shuffle=False, num_workers=2)

sum_ = torch.zeros(3)
sum_sq = torch.zeros(3)
total_pixels = 0

for imgs in base_loader:
    batch_pixels = imgs.size(0) * imgs.size(2) * imgs.size(3)
    total_pixels += batch_pixels
    sum_ += imgs.sum(dim=[0, 2, 3])
    sum_sq += (imgs ** 2).sum(dim=[0, 2, 3])

mean_pre = sum_ / total_pixels
std_pre = torch.sqrt(sum_sq / total_pixels - mean_pre**2)

print("Pre-normalization mean:", mean_pre)
print("Pre-normalization std :", std_pre)

# 3. Transformation including normalization
norm_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean_pre.tolist(), std_pre.tolist())
])

# 4. Loader for post-normalization checks
norm_ds = UnlabeledImageDataset(root, transform=norm_transform)
norm_loader = DataLoader(norm_ds, batch_size=64, shuffle=False, num_workers=2)

# 5. One-batch check
imgs = next(iter(norm_loader))
print("One-batch post-normalization mean:", imgs.mean(dim=[0, 2, 3]))
print("One-batch post-normalization std :", imgs.std(dim=[0, 2, 3]))

# 6. Full dataset post-normalization stats
sum_norm = torch.zeros(3)
sum_norm_sq = torch.zeros(3)
total_pixels = 0

for imgs in norm_loader:
    batch_pixels = imgs.size(0) * imgs.size(2) * imgs.size(3)
    total_pixels += batch_pixels
    sum_norm += imgs.sum(dim=[0, 2, 3])
    sum_norm_sq += (imgs ** 2).sum(dim=[0, 2, 3])

mean_post = sum_norm / total_pixels
std_post = torch.sqrt(sum_norm_sq / total_pixels - mean_post**2)

print("Post-normalization mean (whole dataset):", mean_post)
print("Post-normalization std  (whole dataset):", std_post)

Pre-normalization mean: tensor([0.2554, 0.2554, 0.2554])
Pre-normalization std : tensor([0.2900, 0.2900, 0.2900])
One-batch post-normalization mean: tensor([-0.4267, -0.4267, -0.4267])
One-batch post-normalization std : tensor([0.8331, 0.8331, 0.8331])
Post-normalization mean (whole dataset): tensor([-5.6691e-08, -5.6691e-08, -5.6691e-08])
Post-normalization std  (whole dataset): tensor([1.0000, 1.0000, 1.0000])


10) Let us do Domain Adaptation with DANN using EfficientNet's feature extractor.

In [ ]:
# DANN with EfficientNet-B0 feature extractor

import os, random, numpy as np, pandas as pd, math
from PIL import Image
from tqdm import tqdm

import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets

import matplotlib.pyplot as plt, seaborn as sns
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, roc_auc_score

# ─── Reproducibility ──────────────────────────────────────
SEED = 1906525
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ─── Paths (EfficientNet) ─────────────────────────────────
SRC_DIR = "/content/drive/MyDrive/Research Project 2025/Preprocessed Datasets/Samples/Source - MedMNIST Labelled/Images"
CKPT_DIR = "/content/drive/MyDrive/Research Project 2025/Results/Samples/EfficientNet/Domain Adaptation/Top-5 Source Checkpoints"
TGT_IMG_ROOT = "/content/drive/MyDrive/Research Project 2025/Preprocessed Datasets/Samples/Target - VS Unlabelled/Images"
TGT_CSV = "/content/drive/MyDrive/Research Project 2025/Preprocessed Datasets/Samples/Target - VS Unlabelled/Target sample labels.csv"

CONF_DIR = "/content/drive/MyDrive/Research Project 2025/Results/Samples/EfficientNet/Domain Adaptation/EfficientNet-DANN/Cm_New"
METRICS_DIR = "/content/drive/MyDrive/Research Project 2025/Results/Samples/EfficientNet/Domain Adaptation/EfficientNet-DANN/PM_new"
os.makedirs(CONF_DIR, exist_ok=True); os.makedirs(METRICS_DIR, exist_ok=True)

# ─── Normalization (per-branch) ──────────────────────────
mean_src = [0.3542, 0.3542, 0.3542]; std_src  = [0.2794, 0.2794, 0.2794]
mean_tgt = [0.2554, 0.2554, 0.2554]; std_tgt  = [0.2900, 0.2900, 0.2900]

tx_src = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean_src, std_src)])
tx_tgt = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean_tgt, std_tgt)])

# ─── Import / define EfficientNet‑B0 backbone ────────────
# Assumes EfficientNetB0 class is defined in the same notebook/file.
# If it's in another cell/file, ensure it is imported before running this cell.
# from module import EfficientNetB0

# ---- Minimal wrapper that exposes 1280-d pooled features
class EfficientNetB0_FE(nn.Module):
    def __init__(self, net):
        super().__init__()
        self.net = net
    def forward(self, x):
        # returns (B, 1280) pooled feature vector
        return self.net.forward_features(x)

def load_efficientnet_features(ckpt_path, num_classes):
    # Instantiate our architecture and load weights, dropping classifier.*
    fe_full = EfficientNetB0(num_classes=num_classes)
    sd = torch.load(ckpt_path, map_location='cpu')

    # Drop head weights if present so only feature extractor loads
    for k in list(sd.keys()):
        if k.startswith('classifier.'):
            sd.pop(k, None)
    missing, unexpected = fe_full.load_state_dict(sd, strict=False)
    print(f"[load_efficientnet_features] missing={len(missing)} unexpected={len(unexpected)}")
    return EfficientNetB0_FE(fe_full)

# ─── GRL (λ scales reversed gradient; λ is NOT multiplied on loss) ────────
from torch.autograd import Function
class GradReverse(Function):
    @staticmethod
    def forward(ctx, x, lambd):
        ctx.lambd = lambd
        return x.view_as(x)
    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.neg() * ctx.lambd, None
def grad_reverse(x, lambd=1.0): return GradReverse.apply(x, lambd)

# ─── DANN head (EfficientNet features → bottleneck → task/domain heads) ───
class DANN_EfficientNet(nn.Module):
    def __init__(self, features, num_classes, bottleneck_dim=256, p_drop=0.2):
        super().__init__()
        self.features = features           # returns (B,1280)
        self.bottleneck = nn.Sequential(
            nn.Linear(1280, bottleneck_dim),
            nn.BatchNorm1d(bottleneck_dim),
            nn.ReLU(True),
        )
        self.cls_head = nn.Sequential(
            nn.Dropout(p_drop),
            nn.Linear(bottleneck_dim, num_classes)
        )
        self.dom_head = nn.Sequential(
            nn.Linear(bottleneck_dim, 100), nn.ReLU(True),
            nn.Linear(100, 2)
        )
    def forward(self, x, lambda_=0.0):
        f = self.features(x)               # (B,1280)
        z = self.bottleneck(f)             # (B,256)
        y_logits = self.cls_head(z)        # class logits
        z_rev = grad_reverse(z, lambda_)   # GRL
        d_logits = self.dom_head(z_rev)    # domain logits (2‐class)
        return y_logits, d_logits

# ─── Datasets & loaders ───────────────────────────────────────────────────
# Source labeled
src_ds = datasets.ImageFolder(SRC_DIR, transform=tx_src)
CLASS_NAMES = src_ds.classes
CLASS_TO_IDX = {c:i for i,c in enumerate(CLASS_NAMES)}
N_CLASSES = len(CLASS_NAMES)

def src_loader(bs=32, shuffle=True, num_workers=2):
    return DataLoader(src_ds, batch_size=bs, shuffle=shuffle, num_workers=num_workers, pin_memory=True)

# Target flat folder + CSV with true labels (for evaluation only)
class TargetFlatCSV(Dataset):
    def __init__(self, root, csv_path, transform):
        self.root = root; self.transform = transform
        df = pd.read_csv(csv_path)
        self.names = df.iloc[:,0].astype(str).tolist()
        self.cls_names = df.iloc[:,1].astype(str).tolist()
        self.cls_idx = [CLASS_TO_IDX[c] for c in self.cls_names]
    def __len__(self): return len(self.names)
    def __getitem__(self, i):
        fp = os.path.join(self.root, self.names[i])
        img = Image.open(fp).convert("RGB")
        return self.transform(img), self.cls_idx[i], self.names[i]

tgt_ds = TargetFlatCSV(TGT_IMG_ROOT, TGT_CSV, tx_tgt)

def tgt_loader(bs=32, shuffle=True, num_workers=2):
    return DataLoader(tgt_ds, batch_size=bs, shuffle=shuffle, num_workers=num_workers, pin_memory=True)

# ─── Metrics helpers (unchanged) ──────────────────────────────────────────
def save_confusion_matrix(y_true, y_pred, class_names, save_path):
    cm = confusion_matrix(y_true, y_pred, labels=list(range(len(class_names))))
    plt.figure(figsize=(10,8))
    ax = sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                     xticklabels=class_names, yticklabels=class_names,
                     cbar=False, linewidths=.5)
    ax.set_xlabel('Predicted'); ax.set_ylabel('Actual'); ax.set_title('Confusion Matrix (Target)')
    plt.tight_layout(); plt.savefig(save_path, dpi=220); plt.close()
    return cm

def specificity_from_cm(cm):
    spec = []
    for k in range(cm.shape[0]):
        TP = cm[k,k]
        FP = cm[:,k].sum() - TP
        FN = cm[k,:].sum() - TP
        TN = cm.sum() - (TP+FP+FN)
        spec.append( TN / (TN + FP + 1e-12) )
    return np.array(spec)

def macro_auc(y_true, y_proba, n_classes):
    y_true_oh = np.eye(n_classes)[np.asarray(y_true)]
    return roc_auc_score(y_true_oh, y_proba, average="macro", multi_class="ovr")

# ─── One run: (checkpoint, λ) train + evaluate ────────────────────────────
def dann_train_eval(ckpt_path, ckpt_name, lambda_val, epochs=10, bs=32, lr=1e-4):
    # loaders
    Ls = src_loader(bs=bs, shuffle=True)
    Lt = tgt_loader(bs=bs, shuffle=True)

    # backbone + heads
    feats = load_efficientnet_features(ckpt_path, N_CLASSES)
    model = DANN_EfficientNet(feats, N_CLASSES).to(device)
    opt = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    ce = nn.CrossEntropyLoss()

    model.train()
    it_tgt = iter(Lt)
    for ep in range(epochs):
        for x_s, y_s in Ls:
            x_s, y_s = x_s.to(device), y_s.to(device)
            try:
                x_t, _, _ = next(it_tgt)
            except StopIteration:
                it_tgt = iter(Lt)
                x_t, _, _ = next(it_tgt)
            x_t = x_t.to(device)

            # 1) Task loss on labeled source (no GRL)
            y_logits_s, _ = model(x_s, lambda_=0.0)
            L_cls = ce(y_logits_s, y_s)

            # 2) Domain loss on concat(source,target) with GRL(λ)
            x_dom = torch.cat([x_s, x_t], dim=0)
            _, d_logits = model(x_dom, lambda_=lambda_val)   # GRL scales gradients only
            d_labels = torch.cat([
                torch.zeros(x_s.size(0), dtype=torch.long),
                torch.ones(x_t.size(0), dtype=torch.long)
            ], dim=0).to(device)
            L_dom = ce(d_logits, d_labels)

            loss = L_cls + L_dom
            opt.zero_grad(); loss.backward(); opt.step()

    # ── Evaluate on ALL target images (order from dataset)
    model.eval()
    all_true, all_pred, all_prob = [], [], []
    with torch.no_grad():
        for xb, yb, _names in DataLoader(tgt_ds, batch_size=64, shuffle=False, num_workers=2, pin_memory=True):
            xb = xb.to(device)
            y_logits, _ = model(xb, lambda_=0.0)
            probs = torch.softmax(y_logits, dim=1)
            pred = probs.argmax(dim=1)
            all_true.extend(yb.tolist())
            all_pred.extend(pred.cpu().tolist())
            all_prob.append(probs.cpu().numpy())
    all_prob = np.concatenate(all_prob, axis=0)

    # ── Metrics (same fields/order as before)
    correct = int(np.sum(np.array(all_true)==np.array(all_pred)))
    total = len(all_true)
    acc = correct / total
    miss = 1.0 - acc

    prec, rec, f1, _ = precision_recall_fscore_support(
        all_true, all_pred, labels=list(range(N_CLASSES)),
        average='macro', zero_division=0
    )
    cm = save_confusion_matrix(
        all_true, all_pred, CLASS_NAMES,
        os.path.join(CONF_DIR, f"{ckpt_name}_{lambda_val}.png")
    )
    spec_macro = specificity_from_cm(cm).mean()

    try:
        auc_macro = macro_auc(all_true, all_prob, N_CLASSES)
    except Exception:
        auc_macro = float('nan')

    row = {
        "Checkpoint": ckpt_name,
        "GAN Type": "DANN",
        "DA Hyperparameter": "lambda",
        "DA Hyperparameter Value": lambda_val,
        "Correctly Identified Images": correct,
        "Incorrectly Identified Images": total - correct,
        "Image Classification Accuracy": acc,
        "Image Miss Rate": miss,
        "Precision (macro)": prec,
        "Sensitivity/Recall (macro)": rec,
        "Specificity (macro)": spec_macro,
        "F1-Score (macro)": f1,
        "AUC-ROC (macro OvR)": auc_macro
    }
    return row

# ─── Grid: 5 checkpoints × 5 λ values ─────────────────────────────────────
lambda_vals = [0.01, 0.05, 0.1, 0.5, 1.0]
ckpt_files = [f for f in os.listdir(CKPT_DIR) if f.endswith(".pth")]
results = []

for fname in tqdm(sorted(ckpt_files), desc="Checkpoints x lambdas (EfficientNetB0-DANN)"):
    ckpt_path = os.path.join(CKPT_DIR, fname)
    ckpt_name = os.path.splitext(fname)[0].replace(" ", "").replace("-", "").replace("__", "_")
    for lam in lambda_vals:
        row = dann_train_eval(ckpt_path, ckpt_name, lam, epochs=10, bs=32, lr=1e-4)
        results.append(row)

# ─── Save Excel ───────────────────────────────
df = pd.DataFrame(results)

ordered = (
    ["Checkpoint","GAN Type","DA Hyperparameter","DA Hyperparameter Value"] +
    [c for c in df.columns if c not in ["Checkpoint","GAN Type","DA Hyperparameter","DA Hyperparameter Value"]]
)
df = df[ordered]

save_xlsx = os.path.join(METRICS_DIR, "EfficientNetB0_DANN_metrics.xlsx")
df.to_excel(save_xlsx, index=False)
print(f"✅ Saved metrics to: {save_xlsx}")
print(f"✅ Confusion matrices in: {CONF_DIR}")

Checkpoints x lambdas (EfficientNetB0-DANN):   0%|          | 0/5 [00:00<?, ?it/s]

[load_efficientnet_features] missing=2 unexpected=0
[load_efficientnet_features] missing=2 unexpected=0
[load_efficientnet_features] missing=2 unexpected=0
[load_efficientnet_features] missing=2 unexpected=0
[load_efficientnet_features] missing=2 unexpected=0


Checkpoints x lambdas (EfficientNetB0-DANN):  20%|██        | 1/5 [07:50<31:20, 470.13s/it]

[load_efficientnet_features] missing=2 unexpected=0
[load_efficientnet_features] missing=2 unexpected=0
[load_efficientnet_features] missing=2 unexpected=0
[load_efficientnet_features] missing=2 unexpected=0
[load_efficientnet_features] missing=2 unexpected=0


Checkpoints x lambdas (EfficientNetB0-DANN):  40%|████      | 2/5 [10:56<15:10, 303.51s/it]

...

[load_efficientnet_features] missing=2 unexpected=0
[load_efficientnet_features] missing=2 unexpected=0
[load_efficientnet_features] missing=2 unexpected=0
[load_efficientnet_features] missing=2 unexpected=0
[load_efficientnet_features] missing=2 unexpected=0


Checkpoints x lambdas (EfficientNetB0-DANN):  80%|████████  | 4/5 [17:10<03:44, 224.94s/it]

[load_efficientnet_features] missing=2 unexpected=0
[load_efficientnet_features] missing=2 unexpected=0
[load_efficientnet_features] missing=2 unexpected=0
[load_efficientnet_features] missing=2 unexpected=0
[load_efficientnet_features] missing=2 unexpected=0


Checkpoints x lambdas (EfficientNetB0-DANN): 100%|██████████| 5/5 [20:17<00:00, 243.49s/it]


✅ Saved metrics to: /content/drive/MyDrive/Research Project 2025/Results/Samples/EfficientNet/Domain Adaptation/EfficientNet-DANN/PM_new/EfficientNetB0_DANN_metrics.xlsx
✅ Confusion matrices in: /content/drive/MyDrive/Research Project 2025/Results/Samples/EfficientNet/Domain Adaptation/EfficientNet-DANN/Cm_New


11) Let us do Domain Adaptation with ADDA using EfficientNet's feature extractor.

In [ ]:
# ADDA with EfficientNet-B0 feature extractor
import os, re, random, numpy as np, pandas as pd
from copy import deepcopy
from PIL import Image
from tqdm import tqdm

import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets

import matplotlib.pyplot as plt, seaborn as sns
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, roc_auc_score

# ───────────────────────────── Reproducibility ─────────────────────────────
SEED = 1906525
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ──────────────────────────────── Paths ────────────────────────────────────
SRC_DIR  = "/content/drive/MyDrive/Research Project 2025/Preprocessed Datasets/Samples/Source - MedMNIST Labelled/Images"
CKPT_DIR = "/content/drive/MyDrive/Research Project 2025/Results/Samples/EfficientNet/Domain Adaptation/Top-5 Source Checkpoints"

TGT_IMG_ROOT = "/content/drive/MyDrive/Research Project 2025/Preprocessed Datasets/Samples/Target - VS Unlabelled/Images"
TGT_CSV      = "/content/drive/MyDrive/Research Project 2025/Preprocessed Datasets/Samples/Target - VS Unlabelled/Target sample labels.csv"

CONF_DIR    = "/content/drive/MyDrive/Research Project 2025/Results/Samples/EfficientNet/Domain Adaptation/EfficientNet-ADDA/CM_10"
METRICS_DIR = "/content/drive/MyDrive/Research Project 2025/Results/Samples/EfficientNet/Domain Adaptation/EfficientNet-ADDA/PM_10"
os.makedirs(CONF_DIR, exist_ok=True); os.makedirs(METRICS_DIR, exist_ok=True)

# ─── Normalization (per-branch) ──────────────────────────
mean_src = [0.3542, 0.3542, 0.3542]; std_src  = [0.2794, 0.2794, 0.2794]
mean_tgt = [0.2554, 0.2554, 0.2554]; std_tgt  = [0.2900, 0.2900, 0.2900]

tx_src = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean_src, std_src)
])
tx_tgt = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean_tgt, std_tgt)
])

# ───────────────────── EfficientNet-B0 encoder+head (1280→C) ───────────────
# NOTE: requires our EfficientNetB0 class to be defined/imported above.
class EfficientNetB0Encoder(nn.Module):
    """Return 1280-d GAP-pooled features from a loaded EfficientNetB0."""
    def __init__(self, effb0_full):
        super().__init__()
        # copy modules so src/tgt can diverge
        self.stem   = deepcopy(effb0_full.stem)
        self.blocks = deepcopy(effb0_full.blocks)
        self.head   = deepcopy(effb0_full.head)
        self.pool   = effb0_full.pool  # no params

    def forward(self, x):
        x = self.stem(x)
        x = self.blocks(x)
        x = self.head(x)
        x = self.pool(x).flatten(1)  # (N,1280)
        return x

class ClassifierHead1280(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.fc = nn.Linear(1280, num_classes)
    def forward(self, z):
        return self.fc(z)

class Discriminator(nn.Module):
    # Domain: source=1, target=0  (BCEWithLogits)
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1280, 1024), nn.ReLU(True), nn.Dropout(0.5),
            nn.Linear(1024, 1024), nn.ReLU(True), nn.Dropout(0.5),
            nn.Linear(1024, 1)
        )
    def forward(self, z):
        return self.net(z).view(-1)

# ───────────────────── Load src encoder+classifier from ckpt ───────────────
def load_src_encoder_and_classifier(ckpt_path, num_classes):
    # Rebuild full EfficientNetB0, load weights strictly
    full = EfficientNetB0(num_classes=num_classes)
    sd = torch.load(ckpt_path, map_location='cpu')
    full.load_state_dict(sd, strict=True)

    # Split into encoder (features) and classifier head
    encoder = EfficientNetB0Encoder(full)
    classifier = ClassifierHead1280(num_classes)
    classifier.fc.load_state_dict(full.classifier.state_dict())

    return encoder.to(device), classifier.to(device)

# ───────────────────────────── Datasets & Loaders ──────────────────────────
src_ds = datasets.ImageFolder(SRC_DIR, transform=tx_src)
CLASS_NAMES = src_ds.classes
CLASS_TO_IDX = {c:i for i,c in enumerate(CLASS_NAMES)}
N_CLASSES = len(CLASS_NAMES)

def src_loader(bs=32, shuffle=True, num_workers=2):
    return DataLoader(src_ds, batch_size=bs, shuffle=shuffle, num_workers=num_workers, pin_memory=True)

class TargetFlatCSV(Dataset):
    # CSV: [image_name, class_name]
    def __init__(self, root, csv_path, transform):
        self.root = root; self.transform = transform
        df = pd.read_csv(csv_path)
        self.names = df.iloc[:,0].astype(str).tolist()
        self.cls_names = df.iloc[:,1].astype(str).tolist()
        self.cls_idx = [CLASS_TO_IDX[c] for c in self.cls_names]
    def __len__(self): return len(self.names)
    def __getitem__(self, i):
        fp = os.path.join(self.root, self.names[i])
        img = Image.open(fp).convert("RGB")
        return self.transform(img), self.cls_idx[i], self.names[i]

tgt_ds = TargetFlatCSV(TGT_IMG_ROOT, TGT_CSV, tx_tgt)

def tgt_loader(bs=32, shuffle=True, num_workers=2):
    return DataLoader(tgt_ds, batch_size=bs, shuffle=shuffle, num_workers=num_workers, pin_memory=True)

# ──────────────────────────────── Metrics utils ────────────────────────────
def save_confusion_matrix(y_true, y_pred, class_names, save_path):
    cm = confusion_matrix(y_true, y_pred, labels=list(range(len(class_names))))
    plt.figure(figsize=(10,8))
    ax = sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                     xticklabels=class_names, yticklabels=class_names,
                     cbar=False, linewidths=.5)
    ax.set_xlabel('Predicted'); ax.set_ylabel('Actual'); ax.set_title('Confusion Matrix (Target)')
    plt.tight_layout(); plt.savefig(save_path, dpi=220); plt.close()
    return cm

def specificity_from_cm(cm):
    spec = []
    for k in range(cm.shape[0]):
        TP = cm[k,k]
        FP = cm[:,k].sum() - TP
        FN = cm[k,:].sum() - TP
        TN = cm.sum() - (TP+FP+FN)
        spec.append( TN / (TN + FP + 1e-12) )
    return np.array(spec)

def macro_auc(y_true, y_proba, n_classes):
    # y_proba: (N, C) softmax probs
    y_true_oh = np.eye(n_classes)[np.asarray(y_true)]
    return roc_auc_score(y_true_oh, y_proba, average="macro", multi_class="ovr")

# ─────────────────────────────── ADDA Training ─────────────────────────────
def infinite(dl):
    while True:
        for b in dl: yield b

def lr_name(lr):
    mapping = {1e-3:"1e-3", 5e-4:"5e-4", 2e-4:"2e-4", 1e-4:"1e-4", 5e-5:"5e-5"}
    return mapping.get(lr, f"{lr:.0e}".replace("e-0","e-"))

def sanitize(s):
    s = s.replace(" ", "_")
    s = re.sub(r"[^A-Za-z0-9_]+", "", s)
    return s

@torch.no_grad()
def eval_on_target(tgt_encoder, src_classifier, bs=64):
    tgt_encoder.eval(); src_classifier.eval()
    y_true, y_pred, probs = [], [], []
    for xb, yb, _ in DataLoader(tgt_ds, batch_size=bs, shuffle=False, num_workers=2, pin_memory=True):
        xb = xb.to(device)
        feats = tgt_encoder(xb)         # (N,1280)
        logits = src_classifier(feats)  # (N,C)
        p = torch.softmax(logits, dim=1)
        pred = p.argmax(dim=1)

        y_true.extend(yb.tolist())
        y_pred.extend(pred.cpu().tolist())
        probs.append(p.cpu().numpy())

    probs = np.concatenate(probs, axis=0) if len(probs)>0 else np.zeros((0, N_CLASSES))
    return y_true, y_pred, probs

def train_adda_for_ckpt_lr(ckpt_path, ckpt_name, lr, epochs=10, bs=32):
    # Phase A (done offline): load source encoder & classifier; freeze them
    src_encoder, src_classifier = load_src_encoder_and_classifier(ckpt_path, N_CLASSES)
    for p in list(src_encoder.parameters()) + list(src_classifier.parameters()):
        p.requires_grad_(False)
    src_encoder.eval(); src_classifier.eval()

    # Phase B (ADDA): initialize target encoder from source; adversarially adapt (untied)
    tgt_encoder = deepcopy(src_encoder).train().to(device)
    D = Discriminator().to(device)

    opt_G = optim.Adam(tgt_encoder.parameters(), lr=lr, betas=(0.5, 0.999))
    opt_D = optim.Adam(D.parameters(),           lr=lr, betas=(0.5, 0.999))
    bce   = nn.BCEWithLogitsLoss()

    Ls = src_loader(bs=bs, shuffle=True)
    Lt = tgt_loader(bs=bs, shuffle=True)
    it_s, it_t = infinite(Ls), infinite(Lt)

    steps_per_epoch = min(len(Ls), len(Lt))

    for ep in range(epochs):
        tgt_encoder.train(); D.train()
        loop = tqdm(range(steps_per_epoch), desc=f"[ADDA] {ckpt_name} | lr={lr_name(lr)} | ep {ep+1}/{epochs}", leave=False)
        for _ in loop:
            # ── sample source & target
            xs, ys = next(it_s); xt, _, _ = next(it_t)
            xs = xs.to(device); xt = xt.to(device)

            with torch.no_grad():
                z_s = src_encoder(xs)    # fixed  (N,1280)
            z_t = tgt_encoder(xt)        # trainable

            # ── (1) Train Discriminator: D(z_s)=1, D(z_t)=0
            opt_D.zero_grad(set_to_none=True)
            d_src = D(z_s.detach()); lbl_src = torch.ones_like(d_src)
            d_tgt = D(z_t.detach()); lbl_tgt = torch.zeros_like(d_tgt)
            loss_D = (bce(d_src, lbl_src) + bce(d_tgt, lbl_tgt)) * 0.5
            loss_D.backward(); opt_D.step()

            # ── (2) Train Target Encoder (Generator side): fool D → want D(z_t)=1
            opt_G.zero_grad(set_to_none=True)
            d_tgt_for_g = D(z_t)               # reuse z_t from above
            lbl_trick   = torch.ones_like(d_tgt_for_g)
            loss_G = bce(d_tgt_for_g, lbl_trick)
            loss_G.backward(); opt_G.step()
            loop.set_postfix({"loss_D": float(loss_D), "loss_G": float(loss_G)})

    # ── Evaluate on target using frozen source classifier (csv labels)
    y_true, y_pred, y_prob = eval_on_target(tgt_encoder, src_classifier, bs=64)

    total = len(y_true)
    correct = int(np.sum(np.array(y_true)==np.array(y_pred)))
    acc = correct / (total if total>0 else 1.0)
    miss = 1.0 - acc

    prec, rec, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, labels=list(range(N_CLASSES)), average='macro', zero_division=0
    )

    cm = save_confusion_matrix(
        y_true, y_pred, CLASS_NAMES,
        os.path.join(CONF_DIR, f"{sanitize(ckpt_name)}_{lr_name(lr)}.png")
    )
    spec_macro = specificity_from_cm(cm).mean()

    try:
        auc_macro = macro_auc(y_true, y_prob, N_CLASSES)
    except Exception:
        auc_macro = float('nan')  # handle degenerate cases safely

    row = {
        "Checkpoint": ckpt_name,
        "GAN Type": "ADDA",
        "DA Hyperparameter": "lr",
        "DA Hyperparameter Value": lr_name(lr),
        "Correctly Identified Images": correct,
        "Incorrectly Identified Images": total - correct,
        "Image Classification Accuracy": acc,
        "Classification Rate": acc,
        "Image Miss Rate": miss,
        "Precision (macro)": prec,
        "Sensitivity/Recall (macro)": rec,
        "Specificity (macro)": spec_macro,
        "F1-Score (macro)": f1,
        "AUC-ROC (macro OvR)": auc_macro
    }
    return row

# ───────────────────────────── Run: ckpts × lr grid ────────────────────────
lr_vals = [1e-3, 5e-4, 2e-4, 1e-4, 5e-5]
ckpt_files = [f for f in os.listdir(CKPT_DIR) if f.endswith(".pth")]
results = []

for fname in ckpt_files:
    ckpt_path = os.path.join(CKPT_DIR, fname)
    ckpt_name = os.path.splitext(fname)[0]
    for lr in lr_vals:
        row = train_adda_for_ckpt_lr(ckpt_path, ckpt_name, lr, epochs=10, bs=32)
        results.append(row)

# ───────────────────────────── Save Excel (append) ─────────────────────────
df_new = pd.DataFrame(results)

xlsx_path = os.path.join(METRICS_DIR, "EfficientNetB0_ADDA10_metrics.xlsx")
if os.path.exists(xlsx_path):
    try:
        df_old = pd.read_excel(xlsx_path)
        df_out = pd.concat([df_old, df_new], ignore_index=True)
    except Exception:
        df_out = df_new.copy()
else:
    df_out = df_new.copy()

ordered = (
    ["Checkpoint","GAN Type","DA Hyperparameter","DA Hyperparameter Value"] +
    [c for c in df_out.columns if c not in ["Checkpoint","GAN Type","DA Hyperparameter","DA Hyperparameter Value"]]
)
df_out = df_out[ordered]
df_out.to_excel(xlsx_path, index=False)

print(f"✅ Saved metrics to: {xlsx_path}")
print(f"✅ Confusion matrices saved in: {CONF_DIR}")

✅ Saved metrics to: /content/drive/MyDrive/Research Project 2025/Results/Samples/EfficientNet/Domain Adaptation/EfficientNet-ADDA/PM_10/EfficientNetB0_ADDA10_metrics.xlsx
✅ Confusion matrices saved in: /content/drive/MyDrive/Research Project 2025/Results/Samples/EfficientNet/Domain Adaptation/EfficientNet-ADDA/CM_10
